In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self) -> bytes:
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other) -> bool:
        return tuple(self) == tuple(other)
    
    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)
    
    def __bool__(self) -> bool:
        return bool(abs(self))
    
    def __format__(self, format_spec) -> str:
        if format_spec.endswith('p'):
            format_spec = format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_format = '<{}, {}>'
        else:
            coords = self
            outer_format = '({}, {})'
        components = (format(c, format_spec) for c in coords)
        return outer_format.format(*components)
    
    def angle(self) -> float:
        return math.atan2(self.y, self.x)
    
    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)
    
    def __bool__(self) -> bool:
        return bool(abs(self))


v1 = Vector2d(3, 4)

In [2]:
print(v1)

(3.0, 4.0)
